<h1>Workflow for plotting and visualizing data from the ESAP shopping basket (WSRT-Apertif)</h1>

In [ ]:
from astropy.coordinates import SkyCoord
from hips import WCSGeometry, make_sky_image
from hips import HipsSurveyProperties
from ipywidgets import Layout, Box, widgets
from shopping_client import shopping_client
import pyvo
import pandas as pd
import matplotlib.pyplot as plt
import astroquery

<h2>Fetching data from the ESAP Shopping basket</h2>

<hr>

In [ ]:
# Instantiate alta connector
from alta import alta_connector
ac = alta_connector()

# Instantiate ESAP User Profile shopping client, passing alta connector
shoppingClient = shopping_client(host="https://sdc-dev.astron.nl:443/", connectors=[ac])


In [ ]:
# Get data from the shopping basket as a Pandas Dataframe
basket = shoppingClient.get_basket(convert_to_pandas=True)


In [ ]:
# Store data as a single Pandas Dataframe
alta_records = list(basket["alta"]["record"])
df = pd.DataFrame(alta_records)


In [ ]:
# Show dataframe
df

In [ ]:
# Store data as an astropy table
from astropy.table import Table
astropy_table = Table.from_pandas(df)    

<hr>

# Visualization (Plotting)

## Scatter Plot using Matplotlib

In [ ]:
%matplotlib notebook
from ipywidgets import *
import numpy as np
import matplotlib.pyplot as plt

x = astropy_table["RA"]
y = astropy_table["dec"]
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
line = ax.scatter(x, y)

plt.xlabel("$ra$", fontsize=20)
plt.ylabel("$decl$", fontsize=20)

def update(w = 1.0):
    fig.canvas.draw()

interact(update);

In [ ]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import *
import plotly.graph_objs as go

import numpy as np

init_notebook_mode() # Important

In [ ]:
## Scatter Plot using Plotly

In [ ]:

N = 100000
trace = go.Scattergl(
    x = astropy_table["RA"],
    y = astropy_table["dec"],
    mode = 'markers',
    marker = dict(
        color = '#FFBAD2',
        line = dict(width = 1)
    )
)
data = [trace]
iplot(data, filename='compare_webgl')

## 3d Plot using Plotly

In [ ]:
x, y, z = np.random.multivariate_normal(np.array([0,0,0]), np.eye(3), 200).transpose()
trace1 = go.Scatter3d(
    x=astropy_table["RA"],
    y=astropy_table["dec"],
    z=astropy_table["fov"],
    mode='markers',
    marker=dict(
        size=12,
        line=dict(
            color='rgba(217, 217, 217, 0.14)',
            width=0.5
        ),
        opacity=0.8
    )
)

data = [trace1]
layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)
fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='simple-3d-scatter')

<hr>

# Visualization with Aladin Lite

## Visualize points in the sky

In [ ]:
import ipyaladin as ipyal

In [ ]:
aladin=ipyal.Aladin(target='14 49 4.874 +52 02 9.05', fov=180)
aladin

In [ ]:
aladin.add_table(astropy_table)


## Linked Aladin Lite Views


In [ ]:
# In the following cells, we create 3 linked views of the sky from different surveys & we plot the data points on each one

In [ ]:
from ipywidgets import Layout, Box, widgets
a = ipyal.Aladin(layout=Layout(width='33.33%'), target='14 49 4.874 +52 02 9.05', fov=180)
b = ipyal.Aladin(layout=Layout(width='33.33%'), survey='P/DSS2/red')
c = ipyal.Aladin(layout=Layout(width='33.33%'), survey='P/2MASS/color')

# synchronize target between 3 widgets
widgets.jslink((a, 'target'), (b, 'target'))
widgets.jslink((b, 'target'), (c, 'target'))

# synchronize FoV (zoom level) between 3 widgets
widgets.jslink((a, 'fov'), (b, 'fov'))
widgets.jslink((b, 'fov'), (c, 'fov'))

items = [a, b, c]



box_layout = Layout(display='flex',
                    flex_flow='row',
                    align_items='stretch',
                    border='solid',
                    width='100%')
box = Box(children=items, layout=box_layout)
box


In [ ]:
# Add the table to the views
a.add_table(astropy_table)
b.add_table(astropy_table)
c.add_table(astropy_table)

## Using Aladin Lite to get light curves of objects (using CDS  & TAP)


In [ ]:
aladin = ipyal.Aladin(layout=Layout(width='70%'), target='M 36', fov=0.3)
info = widgets.HTML()


box_layout = Layout(display='flex',
                    flex_flow='row',
                    align_items='stretch',
                    width='100%')
box = Box(children=[aladin, info], layout=box_layout)
box

In [ ]:
aladin.add_table(astropy_table)


In [ ]:

import requests
def process_result(data):
    info.value = ''
    ra = data['ra']
    dec = data['dec']
    radius = min(aladin.fov / 10, 5)
    query = """SELECT TOP 1 main_id, ra, dec, DISTANCE(POINT('ICRS', %f, %f), POINT('ICRS', ra, dec)) as d FROM basic
               WHERE CONTAINS(POINT('ICRS', ra, dec), CIRCLE('ICRS', %f, %f, %f))=1
               ORDER BY d ASC""" % (ra, dec, ra, dec, aladin.fov / 10)
    
    r = requests.get('http://simbad.u-strasbg.fr/simbad/sim-tap/sync', params={'query': query, 'request': 'doQuery', 'lang': 'adql', 'format': 'json', 'phase': 'run'})
    obj_name = ''
    obj_coo = None
    obj_data = r.json()['data']
    if len(obj_data)==0:
        return
    
    obj_name = obj_data[0][0]
    obj_coo = [obj_data[0][1], obj_data[0][2]]
    sed_img = '<img src="http://cdsportal.u-strasbg.fr/cgi-bin/PhotVizPreview/plot?ra=%f&dec=%f&radius_arcsec=5&w=200&h=150&point_size=4">' % (obj_coo[0], obj_coo[1])
    info.value =  '<h2>%s</h2><br><br>%s' % (obj_name, sed_img)
    
aladin.add_listener('click', process_result)


## Using Aladin Lite to get metadata on objects (Single object or Selection)


In [ ]:

from ipyaladin import Aladin
from ipywidgets import Layout, Box, widgets

aladin = Aladin(layout=Layout(width='50%'), target='14 44 11.932 +46 15 32.69', fov=180,survey='http://surveys.roe.ac.uk/hips71/LAS/LAS_Y_OUT//properties')

button = widgets.Button(description="Select")
def on_button_clicked(b):
    aladin.rectangular_selection()

button.on_click(on_button_clicked)
table_info = widgets.HTML(layout=Layout(height='auto', overflow='auto'))


box_layout = Layout(display='flex',
                    flex_flow='row',
                    align_items='stretch',
                    width='100%',
                    position='relative',
                    overflow='hidden',
                    height='100vh',
                    margin='-100px 0 0 0',
                    padding='100px 0 0 0 '
                   )
box = Box(children=[aladin, button, table_info], layout=box_layout)
box

In [ ]:
from astroquery.simbad import Simbad
import astropy.units as u

aladin.add_table(astropy_table)

def process_result(sources):
    s = '<table border="1">'
    s += '<tr><th>RA</th><th>DEC</th></tr>'
    for source in sources:
        s += '<tr><td>%s</td><td>%s</td></tr>' % ( source['data']['RA'], source['data']['dec'])
    s += '</table>'
    table_info.value = s
    
aladin.add_listener('select', process_result)